1.关于特征选择  

简单来说，特征选择就是在你使用机器学习算法之前，通过相关处理来选择与你的预测变量或者输出结果，最有用或最相关的那些特征。它是特征工程的一部分，在机器学习中，我们通常会面临非常多的特征变量，几十个甚至上百个。对于这些特征，一方面全部纳入机器学习算法中会导致计算机开销很大，影响训练效率，另一方面，部分特征并不与预测变量有太大相关，纳入算法中反而会降低模型的准确性，特别是在线性回归、逻辑回归等算法中。   
 
特征选择的好处有：   

减少过度拟合：减少冗余数据意味着根据噪声做出决策的机会减少。  

提高准确度：减少误导性数据意味着提高建模精度。  

缩短训练时间：减少数据意味着算法训练更快。  

2.机器学习中的特征选择方案以及Python举例  

下面以python的sklearn中自带iris鸢尾花数据集为例，来简单演示一些特征选择的案例以及Python实现。该数据集是个二分类问题，且所有属性都是数字。  

（1）单变量选择  

我们可以采用统计中的一些检验来选择与输出变量最相关的那些特征。比如以卡方检验来检验与数据集预测变量（类别）的最佳特征。  

其基本思想：通过观察实际值和理论值的偏差来确定原假设是否成立。首先假设两个变量是独立的（此为原假设），然后观察实际值和理论值之间的偏差程度，若偏差足够小，则认为偏差是很自然的样本误差，接受原假设。若偏差大到一定程度，则否则原假设，接受备择假设。  

在这里采用卡方检验来选择与class关系最强的变量。    
import pandas as pd   
import numpy as np   
from sklearn.feature_selection import SelectKBest #导入SelectKBest库   
from sklearn.feature_selection import chi2 #导入卡方检验 names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']   
dataframe = pd.read_csv("D:\\diabetes.csv", names=names) #读取数据集   
dataframe.head()   
array = dataframe.values   
X = array[:,0:8] #选取前8列为特征变量   
Y = array[:,8] #选取class为目标变量  
print(X[0:5,:]) #打印前5行  
输出结果为：  
[[5.1 3.5 1.4 0.2]  
[4.9 3.  1.4 0.2]  
[4.7 3.2 1.3 0.2]  
[4.6 3.1 1.5 0.2]  
[5.  3.6 1.4 0.2]]  
test = SelectKBest(score_func=chi2, k=2) #设置卡方检验，选择k=2个最佳特征  
fit = test.fit(X, y) #进行检验   
print(fit.scores_) #打印卡方检验值   
features = fit.transform(X) #返回选择特征后的数据   
print(features[0:2,:]) #打印前2行  
输出结果为：  
[ 10.81782088   3.59449902 116.16984746  67.24482759]
[[1.4 0.2]
[1.4 0.2]]  

可以看到筛选X的最后两列为最优变量。  

（2）递归特征消除法  

递归消除特征法使用一个基模型来进行多轮训练，每轮训练后，消除若干权值系数的特征，再基于新的特征集进行下一轮训练。它使用模型精度来识别哪些属性（和属性组合）对预测目标属性的贡献最大。可以通过sklearn库中的RFE来实现。  
from sklearn.feature_selection import RFE #导入RFE库   
from sklearn.linear_model import LogisticRegression #导入逻辑回归库  
model = LogisticRegression() #设置算法为逻辑回归   
rfe = RFE(model, 2) #选择2个最佳特征变量，并进行RFE   
fit = rfe.fit(X, y) #进行RFE递归   
print(fit.n_features_) #打印最优特征变量数   
print( fit.support_) #打印选择的最优特征变量   
print(fit.ranking_) #特征消除排序  
输出结果为：  
2  
[False  True False  True]  
[3 1 2 1]  

可以看到X变量的第2、第4列选为最优变量，即True。最后的[3 1 2 1]也是说明第2、第4列保留到最后。  

（3）主成分分析法  
 
主成分分析法是一种降维技术，不同于前边两种方法，他不消除变量，而是通过变量之间的方差、协方差，吸收最有效信息，通过变量线性组合而成，原转换为维度较小的几个变量。可以通过sklearn库中的PCA来实现。  
from sklearn.decomposition import PCA #导入主成分分析库   
pca = PCA(n_components=2) #提取出2个变量   
fit = pca.fit(X) #模型训练   
print(fit.explained_variance_ratio_) #方差贡献率   
print(fit.components_) #得到的主成分  
输出结果为：  
[0.92461621 0.05301557]
[[ 0.36158968 -0.08226889  0.85657211  0.35884393]
[ 0.65653988  0.72971237 -0.1757674  -0.07470647]]
[0.92461621 0.05301557]是X变量转换为2个新变量的方差贡献率，下边两行的得到两个新变量的载荷矩阵。  

（4）基于树模型的特征选择  

可以采用随机森林的算法，通过树的模型训练可以计算每一个属性的重要性。重要性的值可以帮助我们选择出重要的特征。sklearn.ensemble模块包含了两种基于随机决策树的平均算法：RandomForest算法和Extra-Trees算法。这里使用Extra-Trees算法。  
from sklearn.ensemble import ExtraTreesClassifier #导入ExtraTrees   
model = ExtraTreesClassifier() #设置ExtraTrees   
model.fit(X, y)   
print(model.feature_importances_) #得到特征变量的重要性值  
输出结果为：  
[0.02757014 0.04987713 0.58655833 0.33599439]  

我们可以根据不同特征变量的重要性的值，来选择最佳变量。  

以上是一些特征选择的简单举例，在实际应用中还有很多其他方法，比如相关系数法、互信息法等等。另外，对于鸢尾花数据集来说，其变量属性并不多，且每个变量都有其作用，所以在实际中还需结合更多情况去应用。  

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest #导入SelectKBest库
from sklearn.feature_selection import chi2 #导入卡方检验 names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
# dataframe = pd.read_csv("D:\diabetes.csv", names=names) #读取数据集
# dataframe.head()
# array = dataframe.values
# X = array[:,0:8] #选取前8列为特征变量
# Y = array[:,8] #选取class为目标变量
# print(X[0:5,:]) #打印前5行